<a href="https://colab.research.google.com/github/evanmahmud/Flan-T5/blob/main/Summarization_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate scikit-learn rouge_score peft

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.0 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=cfb50ff205877eab1cf36c59310088210e952ea5fb9d4a8bb0faa99ae8d79172
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's de

In [ ]:
from datasets import Dataset, DatasetDict
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments
import evaluate
import torch
from peft import LoraConfig, get_peft_model

In [ ]:

# Clear GPU cache
torch.cuda.empty_cache()

In [ ]:
'''pip install fastai'''

'pip install fastai'

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
prithwirajsust_bengali_news_summarization_dataset_path = kagglehub.dataset_download('prithwirajsust/bengali-news-summarization-dataset')

print('Data source import complete.')

100%|██████████| 2.12M/2.12M [00:01<00:00, 1.97MB/s]

Extracting files...


Data source import complete.


In [ ]:
print(prithwirajsust_bengali_news_summarization_dataset_path)

/root/.cache/kagglehub/datasets/prithwirajsust/bengali-news-summarization-dataset/versions/1


In [ ]:
import os

for dirname, _, filenames in os.walk('/root/.cache/kagglehub/datasets/prithwirajsust/bengali-news-summarization-dataset/versions/1'):
    print(f"Directory: {dirname}")
    for filename in filenames:
        print(f" - {filename}")

Directory: /root/.cache/kagglehub/datasets/prithwirajsust/bengali-news-summarization-dataset/versions/1
Directory: /root/.cache/kagglehub/datasets/prithwirajsust/bengali-news-summarization-dataset/versions/1/Bengali-News-Summarization-Dataset
 - article.txt
 - summary.txt


In [ ]:
# Define file paths
article_path = os.path.join(prithwirajsust_bengali_news_summarization_dataset_path, 'Bengali-News-Summarization-Dataset/article.txt')
summary_path = os.path.join(prithwirajsust_bengali_news_summarization_dataset_path, 'Bengali-News-Summarization-Dataset/summary.txt')

# Read the article file
with open(article_path, 'r', encoding='utf-8') as f:
    articles = f.read().splitlines()

# Read the summary file
with open(summary_path, 'r', encoding='utf-8') as f:
    summaries = f.read().splitlines()

# Example: Print the first 5 articles and summaries
print("Articles:", articles[:5])
print("Summaries:", summaries[:5])


Articles: ['স্ট্যান্ডার্ড চার্টার্ড ব্যাংকের নতুন প্রধান নির্বাহী কর্মকর্তা হিসেবে দায়িত্ব পেয়েছেন আবরার এ আনোয়ার।', 'রাজধানী থেকে চামড়া শিল্পগুলো সাভারে স্থানান্তরে সিইটিপি ছাড়া সরকার সব কাজ শেষ করেছে বলে জানিয়েছেন শিল্পমন্ত্রী আমির হোসেন আমু।', 'দেশীয় শিল্প বিকাশে সরকারের সব ধরনের উদ্যোগ অব্যাহত রাখার আশ্বাস দিয়েছেন শিল্পমন্ত্রী আমির হোসেন আমু।', 'একীভূত হতে চলেছে অনলাইনে শ্রেণিবদ্ধ বিজ্ঞাপন সেবাদাতা দুই প্রতিষ্ঠান এখানেই ডটকম এবং ওএলএক্স।', 'যাত্রীবাহী একটি বাসে আগুন দেওয়ার আধা ঘণ্টার মধ্যে এই ঘটনায় জড়িত অভিযোগে নড়াইলের পৌর মেয়র ও জেলা বিএনপির সাংগঠনিক সম্পাদক জুলফিকার আলীকে আটক করেছে পুলিশ।']
Summaries: ['স্ট্যান্ডার্ড চার্টার্ডের নতুন সিইও আবরার', 'মার্চের মধ্যে সাভারে চামড়া শিল্পের সিইটিপি: মন্ত্রী', 'ওয়ালটন কারখানায় শিল্পমন্ত্রী', 'একীভূত হচ্ছে এখানেই ডটকমওএলএক্স', 'বাসে আগুন: নড়াইলের পৌর মেয়র গ্রেপ্তার']


In [ ]:
import pandas as pd
df = pd.DataFrame(list(zip(articles,summaries)),columns=["article","summary"])

In [ ]:
df.head()

,article,summary
0,স্ট্যান্ডার্ড চার্টার্ড ব্যাংকের নতুন প্রধান ন...,স্ট্যান্ডার্ড চার্টার্ডের নতুন সিইও আবরার
1,রাজধানী থেকে চামড়া শিল্পগুলো সাভারে স্থানান্তর...,মার্চের মধ্যে সাভারে চামড়া শিল্পের সিইটিপি: মন...
2,দেশীয় শিল্প বিকাশে সরকারের সব ধরনের উদ্যোগ অব্...,ওয়ালটন কারখানায় শিল্পমন্ত্রী
3,একীভূত হতে চলেছে অনলাইনে শ্রেণিবদ্ধ বিজ্ঞাপন স...,একীভূত হচ্ছে এখানেই ডটকমওএলএক্স
4,যাত্রীবাহী একটি বাসে আগুন দেওয়ার আধা ঘণ্টার মধ...,বাসে আগুন: নড়াইলের পৌর মেয়র গ্রেপ্তার


In [ ]:
df = df.head(5000)
df.shape

(5000, 2)

In [ ]:
'''from fastai.text.all import *
# Create DataLoaders
dls = TextDataLoaders.from_df(df, text_col='article', is_lm=True, valid_pct=0.2)

# Fine-tune the language model
learn = language_model_learner(dls, AWD_LSTM, metrics=[accuracy, Perplexity()])
learn.fit_one_cycle(1, 1e-2)
learn.save('bengali_lm')'''

"from fastai.text.all import *\n# Create DataLoaders\ndls = TextDataLoaders.from_df(df, text_col='article', is_lm=True, valid_pct=0.2)\n\n# Fine-tune the language model\nlearn = language_model_learner(dls, AWD_LSTM, metrics=[accuracy, Perplexity()])\nlearn.fit_one_cycle(1, 1e-2)\nlearn.save('bengali_lm')"

In [ ]:
# Load your dataset (replace `df` with your actual DataFrame)
dataset = Dataset.from_dict(df)

# Split the dataset into train, validation, and test sets
dataset = dataset.train_test_split(test_size=0.2, seed=42)  # 80% train, 20% test
train_test_valid = dataset['train'].train_test_split(test_size=0.25, seed=42)  # 60% train, 20% validation, 20% test

# Create a DatasetDict
dataset_dict = DatasetDict({
    'train': train_test_valid['train'],
    'validation': train_test_valid['test'],
    'test': dataset['test']
})

print(dataset_dict)


DatasetDict({
    train: Dataset({
        features: ['article', 'summary'],
        num_rows: 3000
    })
    validation: Dataset({
        features: ['article', 'summary'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['article', 'summary'],
        num_rows: 1000
    })
})


In [ ]:
# Load model and tokenizer
model_name = "google/flan-t5-small"  # Replace with the correct model name if available
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Apply LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)
model = get_peft_model(model, lora_config)


In [ ]:
# Enable gradient checkpointing
model.gradient_checkpointing_enable()


In [ ]:


''' #Tokenize function
def tokenize_function(examples):
    inputs = tokenizer(examples["article"], truncation=True, padding="max_length", max_length=512)
    targets = tokenizer(examples["summary"], truncation=True, padding="max_length", max_length=150)
    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply tokenization to all splits
tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

print(tokenized_datasets)'''


# Tokenize function
def tokenize_function(examples):
    # Tokenize input (article)
    inputs = tokenizer(
        examples["article"],
        truncation=True,
        padding="max_length",
        max_length=128,  # Adjust as needed
    )

    # Tokenize target (summary)
    targets = tokenizer(
        examples["summary"],
        truncation=True,
        padding="max_length",
        max_length=32,  # Adjust as needed
    )

    # Assign labels as lists of token IDs (not tensors)
    inputs["labels"] = targets["input_ids"]
    return inputs

# Tokenize the dataset
tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_datasets["train"][0])

{'article': 'বিশ্বকাপে অতিরিক্ত স্লেজিং দমন করতে কঠোর হবে ক্রিকেটের সর্বোচ্চ সংস্থা আইসিসি। এই অপরাধের জন্য ক্রিকেটাররা নিষিদ্ধ পর্যন্ত হতে পারেন।', 'summary': 'বিশ্বকাপে স্লেজিং করলে শাস্তি নিষেধাজ্ঞাও', 'input_ids': [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True,  # Mixed precision training
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none"
)

In [ ]:
# Load ROUGE metric
rouge = evaluate.load("rouge")

# Define compute_metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute ROUGE scores
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return {k: round(v, 4) for k, v in result.items()}


In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


<ipython-input-20-e7df283feaa3>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
!nvidia-smi

Tue Feb 11 15:13:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P0             26W /   70W |     426MiB /  15360MiB |      6%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Tue Feb 11 15:13:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P0             26W /   70W |     426MiB /  15360MiB |      6%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Train the model
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [ ]:
# Evaluate on the test set
test_results = trainer.evaluate(tokenized_datasets["test"])
print("Test Results:", test_results)

In [ ]:
import pandas as pd

# Generate predictions for the test dataset
test_predictions = trainer.predict(tokenized_datasets["test"])

# Decode the predictions and labels
predicted_summaries = tokenizer.batch_decode(test_predictions.predictions, skip_special_tokens=True)
true_summaries = tokenizer.batch_decode(test_predictions.label_ids, skip_special_tokens=True)
articles = tokenized_datasets["test"]["article"]

# Create a DataFrame with the results
results_df = pd.DataFrame({
    "Article": articles,
    "True Summary": true_summaries,
    "Predicted Summary": predicted_summaries
})

# Save to a CSV file
results_df.to_csv("test_results.csv", index=False)
print("Test results saved to 'test_results.csv'")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Tokenize the true and predicted summaries
true_tokens = [tokenizer.tokenize(summary) for summary in true_summaries]
predicted_tokens = [tokenizer.tokenize(summary) for summary in predicted_summaries]

# Flatten the tokens for metric calculation
true_tokens_flat = [token for sublist in true_tokens for token in sublist]
predicted_tokens_flat = [token for sublist in predicted_tokens for token in sublist]

# Calculate metrics
accuracy = accuracy_score(true_tokens_flat, predicted_tokens_flat)
precision = precision_score(true_tokens_flat, predicted_tokens_flat, average='weighted')
recall = recall_score(true_tokens_flat, predicted_tokens_flat, average='weighted')
f1 = f1_score(true_tokens_flat, predicted_tokens_flat, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)